In [1]:
%load_ext lab_black
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import glob
import tqdm
import umap
import lightgbm as lgb
import numpy as np

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

from utils import DATA_SETS, gather_df, prepare_df, model_function

In [2]:
DATA_SETS

['aquifer_auser',
 'water_spring_amiata',
 'aquifer_petrignano',
 'aquifer_doganella',
 'aquifer_luco',
 'river_arno',
 'lake_bilancino',
 'water_spring_lupa',
 'water_spring_madonna_di_canneto']

In [ ]:
import optuna
import json

result_folder = "sim-res-final"

info_file_name = "./{}/info.json".format(result_folder)
if os.path.exists(info_file_name):
    with open(info_file_name) as f:
        info = json.loads(f.read())
else:
    info = []

for dataset in DATA_SETS:
    df = gather_df(dataset, True)
    target_cols = [c for c in df.columns if "target" in c]

    for target_col in target_cols:
        for pred_ahead in [14, 28, 56]:
            # compute the size of location_matrix_W, which is variable per dataset
            n_feats = (
                len([c for c in df.columns if "ws10m_max" in c])
                + len([c for c in df.columns if "rainfall" in c])
                + len([c for c in df.columns if "temperature" in c])
            )

            # this is the optimization function that we are going to hyperoptimize
            def objective(trial, return_dataframes=False):
                location_array_W = []
                for i in range(n_feats):
                    location_array_W.append(trial.suggest_float("W_{}".format(i), 0, 1))
                dfp_val, dfp_test, lgb_model = model_function(
                    dataset,
                    location_array_W,
                    pred_ahead,
                    target_col,
                    extended_data=trial.suggest_categorical(
                        "extended_data", [True, False]
                    ),
                    impute_missing=trial.suggest_categorical(
                        "impute_missing", [True, False]
                    ),
                    do_extract=trial.suggest_categorical("do_extract", [True, False]),
                    shift_features=trial.suggest_categorical(
                        "shift_features", [True, False]
                    ),
                    use_early_stopping=trial.suggest_categorical(
                        "use_early_stopping", [True, False]
                    ),
                    lgb_boosting_type=trial.suggest_categorical(
                        "lgb_boosting_type", ["gbdt", "goss", "dart"]
                    ),
                    lgb_num_leaves=trial.suggest_categorical(
                        "lgb_num_leaves", [10, 31, 50]
                    ),
                    lgb_learning_rate=trial.suggest_uniform(
                        "lgb_learning_rate", 0.01, 0.3
                    ),
                    lgb_max_depth=trial.suggest_int(
                        "lgb_max_depth", -1, 10
                    ),  # -1 here means infinite
                )
                if return_dataframes:
                    return dfp_val, dfp_test
                return np.mean(
                    np.abs(dfp_val.p - dfp_val.y)
                )  # we optimize on the MAE of the validation dataset

            study = optuna.create_study()
            study.optimize(objective, n_trials=100)

            # the outputting dataframe with optimal parameters
            location_array_W_optim = []
            for k, v in study.best_params.items():
                if "W_" in k:
                    location_array_W_optim.append(v)

            dfp_val, dfp_test, lgb_model = model_function(
                dataset,
                location_array_W_optim,
                pred_ahead,
                target_col,
                extended_data=study.best_params["extended_data"],
                impute_missing=study.best_params["impute_missing"],
                do_extract=study.best_params["do_extract"],
                shift_features=study.best_params["shift_features"],
                use_early_stopping=study.best_params["use_early_stopping"],
                lgb_boosting_type=study.best_params["lgb_boosting_type"],
                lgb_num_leaves=study.best_params["lgb_num_leaves"],
                lgb_learning_rate=study.best_params["lgb_learning_rate"],
                lgb_max_depth=study.best_params["lgb_max_depth"],
            )

            run_id = np.random.randint(1, 100000000000000)

            mae_test = np.mean(np.abs(dfp_test.p - dfp_test.y))
            rmse_test = np.sqrt(np.mean((dfp_test.p - dfp_test.y) ** 2))

            mae_val = np.mean(np.abs(dfp_val.p - dfp_val.y))
            rmse_val = np.sqrt(np.mean((dfp_val.p - dfp_val.y) ** 2))

            info.append(
                {
                    "run_id": run_id,
                    "best_params": study.best_params,
                    "location_weights": location_array_W_optim,
                    "dataset": dataset,
                    "target_col": target_col,
                    "pred_ahead": pred_ahead,
                    "mae_test": mae_test,
                    "rmse_test": rmse_test,
                    "mae_val": mae_val,
                    "rmse_val": rmse_val,
                    "dfp_test_start": str(dfp_test.index[0]),
                    "dfp_test_length": len(dfp_test.index),
                    "mae_normalized_val": (
                        np.abs((dfp_val.y - dfp_val.p))
                        / np.abs(dfp_val.original.mean())
                    ).mean(),
                    "mae_normalized_test": (
                        np.abs((dfp_test.y - dfp_test.p))
                        / np.abs(dfp_test.original.mean())
                    ).mean(),
                }
            )

            with open(info_file_name, "w+") as f:
                json.dump(info, f)
            dfp_val.reset_index().to_feather(
                "./{}/{}-validation.feather".format(result_folder, run_id)
            )
            dfp_test.reset_index().to_feather(
                "./{}/{}-test.feather".format(result_folder, run_id)
            )

[I 2021-02-17 12:34:35,975] Trial 38 finished with value: 3.5398209441702044 and parameters: {'W_0': 0.067957980404099, 'W_1': 0.8183215973636786, 'W_2': 0.029505285881224013, 'W_3': 0.30323337049113824, 'W_4': 0.9071966018204325, 'W_5': 0.8100794436261479, 'extended_data': True, 'impute_missing': True, 'do_extract': False, 'shift_features': False, 'use_early_stopping': False, 'lgb_boosting_type': 'gbdt', 'lgb_num_leaves': 50, 'lgb_learning_rate': 0.19093354064002532, 'lgb_max_depth': -1}. Best is trial 27 with value: 2.21916287595357.
/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
[I 2021-02-17 12:34:36,198] Trial 39 finished with val

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


[I 2021-02-17 12:35:02,886] Trial 0 finished with value: 4.939320620194138 and parameters: {'W_0': 0.7035178965401797, 'W_1': 0.5165996953670825, 'W_2': 0.6939642332039284, 'W_3': 0.5452698789474132, 'W_4': 0.5184357503168293, 'W_5': 0.7271445828856226, 'extended_data': False, 'impute_missing': True, 'do_extract': True, 'shift_features': False, 'use_early_stopping': False, 'lgb_boosting_type': 'goss', 'lgb_num_leaves': 31, 'lgb_learning_rate': 0.19861411696891138, 'lgb_max_depth': 5}. Best is trial 0 with value: 4.939320620194138.
/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
[I 2021-02-17 12:35:03,039] Trial 1 finished with value: 6.

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


[I 2021-02-17 12:35:04,521] Trial 5 finished with value: 4.47427003494198 and parameters: {'W_0': 0.6382188350914204, 'W_1': 0.5487398948684069, 'W_2': 0.1088927715487924, 'W_3': 0.8099971806892486, 'W_4': 0.7675270766645018, 'W_5': 0.7288505176073536, 'extended_data': True, 'impute_missing': True, 'do_extract': False, 'shift_features': True, 'use_early_stopping': False, 'lgb_boosting_type': 'goss', 'lgb_num_leaves': 31, 'lgb_learning_rate': 0.24411694630398334, 'lgb_max_depth': 6}. Best is trial 5 with value: 4.47427003494198.
/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
[I 2021-02-17 12:35:04,693] Trial 6 finished with value: 5.842

...

[I 2021-02-17 12:37:05,862] Trial 37 finished with value: 0.3920414149373964 and parameters: {'W_0': 0.9913319076916176, 'W_1': 0.23298557078201795, 'W_2': 0.2547057574516594, 'W_3': 0.5561929942199716, 'W_4': 0.5395978350745159, 'W_5': 0.07573200506614461, 'extended_data': False, 'impute_missing': False, 'do_extract': True, 'shift_features': True, 'use_early_stopping': True, 'lgb_boosting_type': 'dart', 'lgb_num_leaves': 31, 'lgb_learning_rate': 0.29270954689534745, 'lgb_max_depth': 5}. Best is trial 34 with value: 0.3510685619626499.
/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
[I 2021-02-17 12:37:06,374] Trial 38 finished with val

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
[I 2021-02-17 12:37:37,891] Trial 7 finished with value: 0.5639233132094368 and parameters: {'W_0': 0.6066780546180061, 'W_1': 0.04957358146767643, 'W_2': 0.18488935376697413, 'W_3': 0.7825479717065706, 'W_4': 0.6885255547613455, 'W_5': 0.2839383898766471, 'extended_data': True, 'impute_missing': False, 'do_extract': False, 'shift_features': False, 'use_early_stopping': True, 'lgb_boosting_type': 'dart', 'lgb_num_leaves': 31, 'lgb_learning_rate': 0.29782671364614105, 'lgb_max_depth': 7}. Best is trial 2 with value: 0.4709037996685573.
[I 2021-02-17 12:37:38,356] Trial 8 finished with value

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
[I 2021-02-17 12:37:41,549] Trial 15 finished with value: 0.4980612586844817 and parameters: {'W_0': 0.2592492031812237, 'W_1': 0.5105084751540181, 'W_2': 0.25284770109532084, 'W_3': 0.9071131876753, 'W_4': 0.21006499167432308, 'W_5': 0.6446182813034333, 'extended_data': True, 'impute_missing': True, 'do_extract': True, 'shift_features': False, 'use_early_stopping': True, 'lgb_boosting_type': 'dart', 'lgb_num_leaves': 31, 'lgb_learning_rate': 0.013949230857407902, 'lgb_max_depth': 5}. Best is trial 2 with value: 0.4709037996685573.
/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/ba

In [6]:
with open(info_file_name, "w+") as f:
    json.dump(info, f)

FileNotFoundError: [Errno 2] No such file or directory: './sim-res2/info.json'

In [4]:
dfp_test

,p,y,original
index_col,,,
2018-10-20,-109.311626,-109.51,-109.37
2018-10-21,-109.321621,-109.52,-109.38
2018-10-22,-109.331615,-109.53,-109.39
2018-10-23,-109.341610,-109.54,-109.40
2018-10-24,-109.351605,-109.55,-109.41
...,...,...,...
2020-06-12,-78.588033,-73.93,-78.63
2020-06-13,-78.248214,-73.60,-78.29
2020-06-14,-77.858422,-73.14,-77.90


In [8]:
-1 + 1e-6

-0.999999

In [6]:
dfp_val

,p,y,original
index_col,,,
2017-01-24,-0.0,-103.169905,-103.03
2017-01-25,-0.0,-103.179905,-103.04
2017-01-28,-0.0,-103.209905,-103.07
2017-01-29,-0.0,-103.219905,-103.08
2017-01-30,-0.0,-103.229905,-103.09
...,...,...,...
2018-09-30,-0.0,-109.309934,-109.19
2018-10-01,-0.0,-109.319934,-109.20
2018-10-02,-0.0,-109.329934,-109.21


In [42]:
dfp_val.reset_index().to_feather("./sim-res/{}-validation.feather".format(run_id))
dfp_test.reset_index().to_feather("./sim-res/{}-test.feather".format(run_id))

# info.append({
#     'run_id': run_id,
#     'best_params': best_params,
#     'location_weights': location_array_W_optim,
#     'dataset': dataset,
#     'target_col': target_col,
#     'pred_ahead': pred_ahead
# })
# with open(info_file_name, 'w') as f:
#     json.dump(info, f)